In [2]:
import os
import math
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline



## Data Exploration

In [3]:
df = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
print('Event Rate:', np.mean(df.is_fraud))
display(df.head(3))

**The event rate is 0.5%, so the dataset is highly imbalanced, we will random pick 20% non-event observations and all event observations to do undersampling.**

In [4]:
train_data = df[df.is_fraud==0].sample(frac=0.2, random_state = 2).append(df[df.is_fraud==1])
print('Event Rate:', np.mean(train_data.is_fraud))
print('Event Rate Distribution:\n', train_data.is_fraud.value_counts())


### Prepare Graph Data

In [5]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df = df_input.copy()
    mapping = {x: node_id for node_id, x in enumerate(set(df['cc_num'].values.tolist()+
                                                          df['merchant'].values.tolist()))}
    df['from'] = df['cc_num'].apply(lambda x: mapping[x])
    df['to'] = df['merchant'].apply(lambda x: mapping[x])
    df = df[['from','to','amt','is_fraud']].groupby(['from','to']).agg({'is_fraud':'sum','amt':'sum'}).reset_index()
    df['is_fraud'] = df['is_fraud'].apply(lambda x: 1 if x>0 else 0)
    
    G = nx.from_edgelist(df[['from','to']].values, create_using = graph_type)
    
    nx.set_node_attributes(G, {x:1 for x in df['from'].unique()}, 'bipartite')
    nx.set_node_attributes(G, {x:2 for x in df['to'].unique()}, 'bipartite')
    
    nx.set_edge_attributes(G, 
                          {(int(x['from']), int(x['to'])): x['is_fraud'] for idx, x in df[['from','to','is_fraud']].iterrows()},
                          'label')
    nx.set_edge_attributes(G, 
                          {(int(x['from']), int(x['to'])): x['amt'] for idx, x in df[['from','to','amt']].iterrows()},
                          'weight')
    return(G)

In [6]:
def build_graph_tripartite(df_input, graph_type=nx.Graph()):
    df = df_input.copy()
    mapping = {x: node_id for node_id, x in enumerate(set(df.index.values.tolist() + 
                                                          df['cc_num'].values.tolist()+
                                                          df['merchant'].values.tolist()))}
    
    df['in_node'] = df['cc_num'].apply(lambda x: mapping[x])
    df['out_node'] = df['merchant'].apply(lambda x: mapping[x])
       
    G = nx.from_edgelist([(x['in_node'], mapping[idx]) for idx, x in df.iterrows()] +
                         [(x['out_node'], mapping[idx]) for idx, x in df.iterrows()],
                         create_using = graph_type)
    
    nx.set_node_attributes(G, {x['in_node']:1 for idx, x in df.iterrows()}, 'bipartite')
    nx.set_node_attributes(G, {x['out_node']:2 for idx, x in df.iterrows()}, 'bipartite')
    nx.set_node_attributes(G, {mapping[idx]:3 for idx, x in df.iterrows()}, 'bipartite')
    
    nx.set_edge_attributes(G, 
                          {(int(x['in_node']), mapping[idx]): x['is_fraud'] for idx, x in df.iterrows()},
                          'label')
    nx.set_edge_attributes(G, 
                          {(int(x['out_node']), mapping[idx]): x['is_fraud'] for idx, x in df.iterrows()},
                          'label')
   
    nx.set_edge_attributes(G, 
                          {(int(x['in_node']), mapping[idx]): x['amt'] for idx, x in df.iterrows()},
                          'weight')
    nx.set_edge_attributes(G, 
                          {(int(x['out_node']), mapping[idx]): x['amt'] for idx, x in df.iterrows()},
                          'weight')
    
    return(G)

In [6]:
G_bu = build_graph_bipartite(train_data, nx.Graph(name=['Bipartite Undirected']))
G_bd = build_graph_bipartite(train_data, nx.DiGraph(name=['Bipartite Directed']))
G_tu = build_graph_tripartite(train_data, nx.Graph(name=['Tripartite Undirected']))
G_td = build_graph_tripartite(train_data, nx.DiGraph(name=['Tripartite Directed']))

In [7]:
from networkx.algorithms import bipartite
bipartite.is_bipartite(G_bu)

In [8]:
print(nx.info(G_bu))
print('==========================')
print(nx.info(G_tu))

**Node Degree Distribution**

From the below plot we can see, the bipartite graph has a more variegated distribution, with a peak of around 300. While tripartite graph has a peak of degree 2.

In [9]:
for G in [G_bu, G_tu]:
    plt.figure(figsize = (10,10))
    degrees = pd.Series(
        {
            k:v for k,v in nx.degree(G)
        }
    )
    degrees.plot.hist()
    plt.yscale("log")

**Edge Weight Distribution**

From the below plot, the distribution is slightly shifted to the right (right-skewed) when compared to the tripartite where the transaction nodes are more pronounced.

In [10]:
for G in [G_bu, G_tu]:
    allEdgesWeights = pd.Series({
        (d[0], d[1]): d[2]['weight'] for d in G.edges(data=True)
    })
    np.quantile(allEdgesWeights.values, [0.1, 0.5, 0.7, 0.9, 1])
    quant_dist = np.quantile(allEdgesWeights.values, [0.1, 0.5, 0.7, 0.9])
    allEdgesWeightsFiltered = pd.Series({
        (d[0], d[1]): d[2]['weight'] for d in G.edges(data=True) if d[2]['weight'] < quant_dist[-1]
    })
    plt.figure(figsize = (10,10))
    allEdgesWeightsFiltered.plot.hist(bins=40)
    plt.yscale('log')

**Degree Centrality**

Degree Centrality is defined as the number of links incident upon a node (i.e., the number of ties that a node has). 


In [11]:
for G in [G_bu, G_tu]:
    plt.figure(figsize = (10,10))
    degree_centrality = pd.Series(
        {
            k:v for k,v in nx.degree_centrality(G).items()
        }
    )
    degree_centrality.plot.hist()
    plt.yscale("log")

**Betweenness Centrality**

It measures the number of shortest paths that pass through a given node, providing and intuition about how central that node is for message passing within the network.

In [ ]:
# it takes forever to run this code
# for G in [G_bu, G_tu]:
#     plt.figure(figsize = (10,10))
#     degree_centrality = pd.Series(
#         {
#             k:v for k,v in nx.betweenness_centrality(G).items()
#         }
#     )
#     degree_centrality.plot.hist()
#     plt.yscale("log")

**Closeness Centrality**

It is a way of detecting nodes that are able to spread information very efficiently through a graph. the closeness centrality of a node measures its average farness (inverse distance) to all other nodes. Nodes with a high closeness score have the shortest distances to all other nodes. 

In [12]:
# it takes forever to run this code
# for G in [G_bu, G_tu]:
#     plt.figure(figsize = (10,10))
#     degree_centrality = pd.Series(
#         {
#             k:v for k,v in nx.closeness_centrality(G).items()
#         }
#     )
#     degree_centrality.plot.hist()
#     plt.yscale("log")

**Assortativity**: 

is a preference for a network's nodes to attach to others that are similar in some way. Though the specific measure of similarity may vary, network theorists often examin assotativity interms of a node's degree.

In [13]:
nx.degree_pearson_correlation_coefficient(G_bu)

In [15]:
nx.degree_pearson_correlation_coefficient(G_tu)

### Community Detection & Fraudulent Edges Ratio Analysis

Communities are extracted based on the edge weights using community.best_partition command

In [17]:
# pip install python-louvain
import community
parts = community.best_partition(G_bu, random_state = 4, weight = 'weight')
communities = pd.Series(parts)
communities.value_counts().sort_values(ascending = False)

From the above result, there are 19 communities in total, communities 12 has the highest weight where community 15 has the lowest

In [18]:
communities.value_counts().plot.hist(bins=20)

The following code generates a node-inducedsubgraph by using the nodes present in a particular community. Since in bipartite graph, all the transactions are donoted by the corresponding edges between the nodes. we will take the fraud_edges / tmp.number_of_edges() for computing the fraudulent ratio percentage.

The output of the below code,community 0 has the most fraudulent transaction percentage with 21.6%, closely followed by community 16 and community 14. Community 15 has the least percentage of fraudulent transactions.

In [20]:
graphs = []
d = {}
for x in communities.unique():
    tmp = nx.subgraph(G_bu, communities[communities == x].index)
    fraud_edges = sum(nx.get_edge_attributes(tmp, 'label').values())
    ratio = 0 if fraud_edges == 0 else (fraud_edges/tmp.number_of_edges())*100
    d[x] = ratio
    graphs += [tmp]
    
pd.Series(d).sort_values(ascending = False)

In [29]:
gID = 1
plt.figure(figsize = (10,10))
spring_pos = nx.spring_layout(graphs[gID])
plt.axis('off')
edge_colors = ['r' if x==1 else 'g' for x in nx.get_edge_attributes(graphs[gID], 'label').values()]
nx.draw_networkx(graphs[gID], pos=spring_pos, 
                edge_color=edge_colors, with_labels = True, node_size = 15)

* Community Detection is similar to the Clustering method in unsupervised learning
* Different communities are identified with various levels of density
* All the communities are evaluated for the Ratio of Fraud to Benign transaction edges and shown in plots
## Model Development -- Supervised Learning
### Prepare Dataset (Undersampling and split training/testing)

In [8]:
from sklearn.utils import resample
df_majority = df[df.is_fraud == 0]
df_minority = df[df.is_fraud == 1]
df_maj_dowsampled = resample(df_majority,
                            n_samples = len(df_minority),
                            random_state = 42)
df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
print(df_downsampled.is_fraud.value_counts())
G_down = build_graph_bipartite(df_downsampled)

In [10]:
from sklearn.model_selection import train_test_split
train_edges, test_edges, train_labels, test_labels = train_test_split(list(range(len(G_down.edges))),
                                                                      list(nx.get_edge_attributes(G_down, 'label').values()),
                                                                      test_size = 0.2,
                                                                      random_state = 4
                                                                     )

**Node to Vector Embedding**

In [18]:
!pip install node2vec
!pip install python-Levenshtein
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder



node2vec_train = Node2Vec(G_down, weight_key = 'weight')
model_train = node2vec_train.fit(window = 10)

### Supervised Learning

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

classes = [HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder]
edgs =list(G_down.edges)
for cl in classes:
    embeddings_train = cl(keyed_vectors = model_train.wv)
    
    train_embeddings = [embeddings_train[str(edgs[x][0]), str(edgs[x][1])] for x in train_edges]
    test_embeddings = [embeddings_train[str(edgs[x][0]), str(edgs[x][1])] for x in test_edges]
    
    rf = RandomForestClassifier(n_estimators = 1000, random_state = 4)
    rf.fit(train_embeddings, train_labels)
    
    y_pred = rf.predict(test_embeddings)
    print(cl)
    print('Precision:', metrics.precision_score(test_labels, y_pred))
    print('Recall:', metrics.recall_score(test_labels, y_pred))
    print('F1-Score:', metrics.f1_score(test_labels, y_pred))

### Unsupervised Learning

In [48]:
node2vec_unsup = Node2Vec(G_down, weight_key = 'weight')
unsup_vals = node2vec_unsup.fit(window = 10)

In [51]:
from sklearn.cluster import KMeans

classes = [HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder]
true_labels = [x for x in nx.get_edge_attributes(G_down, 'label').values()]

for cl in classes:
    embedding_edge = cl(keyed_vectors = unsup_vals.wv)
    
    embedding = [embedding_edge[str(x[0]), str(x[1])] for x in G_down.edges()]
    kmeans = KMeans(2, random_state = 4).fit(embedding)
    
    nmi = metrics.adjusted_mutual_info_score(true_labels, kmeans.labels_)
    ho = metrics.homogeneity_score(true_labels, kmeans.labels_)
    co = metrics.completeness_score(true_labels, kmeans.labels_)
    vmeasure = metrics.v_measure_score(true_labels, kmeans.labels_)
    
    print(cl)
    print('NMI:', nmi)
    print('Homogeneity:', ho)
    print('Completeness:', co)
    print('V-Measure:', vmeasure)

for unsupervised KMeans approach, WeightedL1Embedder has the best performance.